In [7]:
import numpy as np
from umap import umap
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

import os, sys
from tqdm.notebook import tqdm
# sys.path.append(r"C:\Users\chataint\Documents\projet\humanlisbet\lisbet_code")
# from utils import load_embedding


In [2]:
data_r = r"C:\Users\chataint\Documents\projet\humanlisbet\results\bet_embedders\bet_embedders\13879972"
seed = 42

In [3]:

datapath = os.path.join(data_r, "embedding_train.numpy")
dataval = os.path.join(data_r, "embedding_test.numpy")

# Load your temporal encoded data (shape: n_samples, 128 dimensions)
train_data =  load_embedding(datapath)# Replace with your data
td = train_data.drop(columns='video').to_numpy()


In [9]:
umap.__dict__

{'__name__': 'umap',
 '__doc__': None,
 '__package__': '',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x1a6b9947460>,
 '__spec__': ModuleSpec(name='umap', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001A6B9947460>, origin='c:\\Users\\chataint\\Documents\\projet\\humanlisbet\\lisbet_code\\embedding_analysis\\umap.py'),
 '__file__': 'c:\\Users\\chataint\\Documents\\projet\\humanlisbet\\lisbet_code\\embedding_analysis\\umap.py',
 '__cached__': 'c:\\Users\\chataint\\Documents\\projet\\humanlisbet\\lisbet_code\\embedding_analysis\\__pycache__\\umap.cpython-38.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>),
  '__build_class__': <function __bui

In [8]:
umap_reducer = umap.UMAP(n_components=2)


AttributeError: module 'umap' has no attribute 'UMAP'

In [4]:

# Step 1: Dimensionality Reduction with UMAP
data_umap = umap_reducer.fit_transform(td)

c:\Users\chataint\AppData\Local\anaconda3\envs\diffversify\lib\site-packages\sklearn\manifold\_spectral_embedding.py:392: UserWarning: Exited at iteration 526 with accuracies 
[1.15224576e-15 2.34491319e-05 2.62052876e-05 2.40631266e-05]
not reaching the requested tolerance 2.4721026420593262e-05.
Use iteration 427 instead with accuracy 
1.509689262591631e-05.

  _, diffusion_map = lobpcg(
c:\Users\chataint\AppData\Local\anaconda3\envs\diffversify\lib\site-packages\sklearn\manifold\_spectral_embedding.py:392: UserWarning: Exited postprocessing with accuracies 
[1.47487181e-15 2.08817564e-05 1.33297035e-05 2.61761107e-05]
not reaching the requested tolerance 2.4721026420593262e-05.
  _, diffusion_map = lobpcg(


KeyboardInterrupt: 

In [ ]:
# Step 2: K-Means Clustering and Finding the Optimal Number of Clusters
def kmeans_clustering(data, max_clusters=10, min_clusters=2, step=1):
    inertia = []
    silhouette = []

    for k in tqdm(range(min_clusters, max_clusters + 1, step)):
        kmeans = KMeans(n_clusters=k, random_state=seed)
        labels = kmeans.fit_predict(data)
        inertia.append(kmeans.inertia_)
        silhouette.append(silhouette_score(data, labels))
    return inertia, silhouette

# Run clustering for up to 10 clusters
min_clusters = 5
max_clusters = 100
step=5
inertia, silhouette = kmeans_clustering(data_umap, max_clusters, min_clusters, step)

In [ ]:
# Step 3: Plot Inertia and Silhouette Scores
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].plot(range(2, max_clusters + 1), inertia, marker='o')
ax[0].set_title("Elbow Method (Inertia)")
ax[0].set_xlabel("Number of Clusters")
ax[0].set_ylabel("Inertia")

ax[1].plot(range(2, max_clusters + 1), silhouette, marker='o')
ax[1].set_title("Silhouette Score")
ax[1].set_xlabel("Number of Clusters")
ax[1].set_ylabel("Silhouette Score")

plt.show()

In [ ]:
# Step 4: Visualization of Clusters
optimal_clusters = silhouette.index(max(silhouette)) + 2
kmeans = KMeans(n_clusters=optimal_clusters, random_state=seed)
labels = kmeans.fit_predict(data_umap)

plt.scatter(data_umap[:, 0], data_umap[:, 1], c=labels, cmap='viridis', s=10)
plt.title(f"K-Means Clustering on UMAP with {optimal_clusters} Clusters")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.colorbar(label="Cluster Label")
plt.show()
